In [1]:
! pip install kneed

In [2]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np
from math import *
from shapely.geometry import Point, Polygon
import geopandas as gpd
import sys
print(sys.path)
import math
from tqdm import tqdm
from pandas import DataFrame
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from kneed import KneeLocator
from sklearn.neighbors import NearestNeighbors, KDTree
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from threadpoolctl import threadpool_limits
from sklearn.preprocessing import StandardScaler

import pymongo
import json
import pandas as pd
from datetime import datetime, timedelta
from pandas import DataFrame
import matplotlib.pyplot as plt
from datetime import time
from tqdm import tqdm

from math import *
from shapely.geometry import Point, Polygon
import geopandas as gpd
import sys
print(sys.path)
import math
from pandas import DataFrame
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from kneed import KneeLocator
from sklearn.neighbors import KDTree
from sklearn.cluster import DBSCAN

['/kaggle/lib/kagglegym', '/kaggle/lib', '/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/root/.local/lib/python3.10/site-packages', '/opt/conda/lib/python3.10/site-packages', '/root/src/BigQuery_Helper']
['/kaggle/lib/kagglegym', '/kaggle/lib', '/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/root/.local/lib/python3.10/site-packages', '/opt/conda/lib/python3.10/site-packages', '/root/src/BigQuery_Helper']


In [3]:
student_uids = ['u00', 'u01', 'u02', 'u03', 'u04', 'u05', 'u07', 'u08', 'u09', 'u10', 'u12', 'u13', 'u14', 'u15', 'u16',
                'u17', 'u18', 'u19', 'u20', 'u22', 'u23', 'u24', 'u25', 'u27', 'u30', 'u31', 'u32', 'u33', 'u34', 'u35',
                'u36', 'u39', 'u41', 'u42', 'u43', 'u44', 'u45', 'u46', 'u47', 'u49', 'u50', 'u51', 'u52', 'u53', 'u54',
                'u56', 'u57', 'u58', 'u59']
print(len(student_uids))

49


In [4]:
df_gps = pd.read_csv('/kaggle/input/studentlife-location-four-intervals/StudentLife_location_data.csv')
df_gps = df_gps.sort_values(by=['uid', 'timestamp'])
df_gps['timestamp'] = pd.to_datetime(df_gps['timestamp'])
del df_gps['Unnamed: 0']
df_gps.head(10)

,uid,timestamp,latitude,longitude,date,time_interval
156575,u00,2013-03-27 04:02:43,43.759135,-72.329240,2013-03-27,night
156576,u00,2013-03-27 04:22:24,43.759503,-72.329018,2013-03-27,night
156577,u00,2013-03-27 04:42:44,43.759134,-72.329238,2013-03-27,night
156578,u00,2013-03-27 05:02:43,43.759135,-72.329240,2013-03-27,night
156579,u00,2013-03-27 05:22:43,43.759135,-72.329240,2013-03-27,night
156580,u00,2013-03-27 05:42:43,43.759135,-72.329239,2013-03-27,night
156581,u00,2013-03-27 06:02:43,43.759134,-72.329241,2013-03-27,morning
156582,u00,2013-03-27 06:22:44,43.608052,-72.162171,2013-03-27,morning
156583,u00,2013-03-27 06:22:48,43.608052,-72.162171,2013-03-27,morning
156584,u00,2013-03-27 07:02:43,43.696293,-72.283302,2013-03-27,morning


In [5]:
features = df_gps[['latitude', 'longitude']].values

scaler = StandardScaler()
features_normalized = scaler.fit_transform(features)

k = 3  
neigh = NearestNeighbors(n_neighbors=k)
neigh.fit(features_normalized)
distances, _ = neigh.kneighbors(features_normalized)

distances = np.sort(distances[:, k-1])

def angle(p1, p2, p3):
    v1 = p1 - p2
    v2 = p3 - p2
    return np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))

x = np.arange(len(distances))
coords = np.column_stack((x, distances))
start, end = coords[0], coords[-1]
angles = [angle(start, p, end) for p in coords]
elbow_index = np.argmax(angles)
optimal_eps = distances[elbow_index]

print(f"Optimal eps value: {optimal_eps}")

/tmp/ipykernel_17/587325574.py:16: RuntimeWarning: invalid value encountered in scalar divide
  return np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))


Optimal eps value: 0.0


In [6]:
# number of different locations visited
# variance in time spent at different locations 

In [7]:
def count_places_visited(df, eps=0.001, min_samples=3):
    results = []
    unique_dates = df['date'].unique()

    for date in unique_dates:
        places_count = {
            'number_places_morning': 0,
            'number_places_afternoon': 0,
            'number_places_evening': 0,
            'number_places_night': 0,
            'time_spent_morning': [],
            'time_spent_afternoon': [],
            'time_spent_evening': [],
            'time_spent_night': [],
            'variance_time_spent_morning': 0,
            'variance_time_spent_afternoon': 0,
            'variance_time_spent_evening': 0,
            'variance_time_spent_night': 0
        }
        
        for time_interval in ['morning', 'afternoon', 'evening', 'night']:
            subset = df[(df['date'] == date) & 
                        (df['time_interval'] == time_interval)].copy()
            
            if not subset.empty:
                coords = subset[['latitude', 'longitude']].values
                db = DBSCAN(eps=eps, min_samples=min_samples).fit(coords)
                labels = db.labels_
                unique_labels = set(labels)
                
                num_places = len(unique_labels) - (1 if -1 in unique_labels else 0)
                places_count[f'number_places_{time_interval}'] = num_places

                # time spent at each location
                for label in unique_labels:
                    if label != -1:
                        location_subset = subset[labels == label].sort_values(by='timestamp')
                        time_spent = 0
                        
                        for i in range(len(location_subset) - 1):
                            if labels[i] == labels[i + 1]:
                                time_spent += (location_subset['timestamp'].iloc[i + 1] - 
                                               location_subset['timestamp'].iloc[i]).total_seconds()
                            else:
                                time_spent += 1

                        time_spent /= 3600  # convert to hours
                        places_count[f'time_spent_{time_interval}'].append(time_spent)

                # variance in time spent at different locations
                if places_count[f'time_spent_{time_interval}']:
                    places_count[f'variance_time_spent_{time_interval}'] = np.var(places_count[f'time_spent_{time_interval}'])
            else:
                places_count[f'number_places_{time_interval}'] = 0
                places_count[f'variance_time_spent_{time_interval}'] = 0
        
        results.append({
            'uid': df['uid'].iloc[0],
            'date': date,
            **places_count
        })
        
    results_df = pd.DataFrame(results)
    del results_df['time_spent_morning']
    del results_df['time_spent_afternoon']
    del results_df['time_spent_evening']
    del results_df['time_spent_night']
    
    return results_df

In [8]:
result_1 = DataFrame()

for uid in tqdm(student_uids):
    print(uid)
    df_ = df_gps[df_gps['uid'] == uid]
    df__ = count_places_visited(df_)
    frames = [result_1, df__]
    result_1 = pd.concat(frames, ignore_index=True)

  0%|          | 0/49 [00:00<?, ?it/s]

u00


  2%|▏         | 1/49 [00:01<01:30,  1.88s/it]

u01


  4%|▍         | 2/49 [00:03<01:29,  1.90s/it]

...

 96%|█████████▌| 47/49 [01:13<00:03,  1.67s/it]

u58


 98%|█████████▊| 48/49 [01:15<00:01,  1.76s/it]

u59


100%|██████████| 49/49 [01:17<00:00,  1.58s/it]


In [9]:
result_1.head(3)

,uid,date,number_places_morning,number_places_afternoon,number_places_evening,number_places_night,variance_time_spent_morning,variance_time_spent_afternoon,variance_time_spent_evening,variance_time_spent_night
0,u00,2013-03-27,1,2,2,1,0.000000,3.362130,0.251252,0.000000
1,u00,2013-03-28,2,2,1,3,0.027870,2.250417,0.000000,0.024733
2,u00,2013-03-29,2,3,2,4,0.759319,1.506152,0.111482,0.020811


In [10]:
print(result_1['number_places_morning'].max())
print(result_1['number_places_afternoon'].max())
print(result_1['number_places_evening'].max())
print(result_1['number_places_night'].max())

4
6
5
6


In [11]:
print(result_1['variance_time_spent_morning'].max())
print(result_1['variance_time_spent_afternoon'].max())
print(result_1['variance_time_spent_evening'].max())
print(result_1['variance_time_spent_night'].max())

5.4444444444444455
6.249999999999999
5.880625
5.840949093364198


In [12]:
result_1['number_places_morning'].value_counts()

number_places_morning
1    2347
0     308
2     149
3      16
4       4
Name: count, dtype: int64

In [13]:
# time spent in mostly visited location
# time spent in second most visited location
# time spent in third most visited location

In [14]:
def count_time_spent_places_visited(df, eps=0.001, min_samples=3):
    results = []
    unique_dates = df['date'].unique()

    for date in unique_dates:
        places_count = {
            'time_spent_morning_firstplace': 0,
            'time_spent_afternoon_firstplace': 0,
            'time_spent_evening_firstplace': 0,
            'time_spent_night_firstplace': 0,
            'time_spent_morning_secondplace': 0,
            'time_spent_afternoon_secondplace': 0,
            'time_spent_evening_secondplace': 0,
            'time_spent_night_secondplace': 0,
            'time_spent_morning_thirdplace': 0,
            'time_spent_afternoon_thirdplace': 0,
            'time_spent_evening_thirdplace': 0,
            'time_spent_night_thirdplace': 0,
        }
        
        for time_interval in ['morning', 'afternoon', 'evening', 'night']:
            subset = df[(df['date'] == date) & 
                        (df['time_interval'] == time_interval)].copy()
            
            if not subset.empty:
                coords = subset[['latitude', 'longitude']].values
                db = DBSCAN(eps=eps, min_samples=min_samples).fit(coords)
                labels = db.labels_
                
                label_counts = pd.Series(labels).value_counts()
                sorted_labels = label_counts.index[label_counts.index != -1][:3]  # first 3 labels
                
                # time spent calculation function
                def calculate_time_spent(location_subset):
                    time_spent = 0
                    location_subset = location_subset.sort_values(by='timestamp')
                    for i in range(len(location_subset) - 1):
                        time_spent += (location_subset['timestamp'].iloc[i + 1] - 
                                       location_subset['timestamp'].iloc[i]).total_seconds()
                    time_spent /= 3600  # convert to hours
                    return time_spent
                
                # first place
                if len(sorted_labels) > 0:
                    location_subset = subset[labels == sorted_labels[0]]
                    places_count[f'time_spent_{time_interval}_firstplace'] = calculate_time_spent(location_subset)
                
                # second place
                if len(sorted_labels) > 1:
                    location_subset = subset[labels == sorted_labels[1]]
                    places_count[f'time_spent_{time_interval}_secondplace'] = calculate_time_spent(location_subset)
                
                # third place
                if len(sorted_labels) > 2:
                    location_subset = subset[labels == sorted_labels[2]]
                    places_count[f'time_spent_{time_interval}_thirdplace'] = calculate_time_spent(location_subset)
        
        results.append({
            'uid': df['uid'].iloc[0],
            'date': date,
            **places_count
        })
        
    results_df = pd.DataFrame(results)
    
    return results_df

In [15]:
result_2 = DataFrame()

for uid in tqdm(student_uids):
    print(uid)
    df_ = df_gps[df_gps['uid'] == uid]
    df__ = count_time_spent_places_visited(df_)
    frames = [result_2, df__]
    result_2 = pd.concat(frames, ignore_index=True)

  0%|          | 0/49 [00:00<?, ?it/s]

u00


  2%|▏         | 1/49 [00:01<01:24,  1.77s/it]

u01


  4%|▍         | 2/49 [00:03<01:30,  1.93s/it]

...

 96%|█████████▌| 47/49 [01:16<00:03,  1.79s/it]

u58


 98%|█████████▊| 48/49 [01:18<00:01,  1.87s/it]

u59


100%|██████████| 49/49 [01:21<00:00,  1.65s/it]


In [16]:
result_2.head(3)

,uid,date,time_spent_morning_firstplace,time_spent_afternoon_firstplace,time_spent_evening_firstplace,time_spent_night_firstplace,time_spent_morning_secondplace,time_spent_afternoon_secondplace,time_spent_evening_secondplace,time_spent_night_secondplace,time_spent_morning_thirdplace,time_spent_afternoon_thirdplace,time_spent_evening_thirdplace,time_spent_night_thirdplace
0,u00,2013-03-27,4.000000,5.333333,2.668611,1.666667,0.000000,1.000000,1.000000,0.000000,0.0,0.000000,0.0,0.000000
1,u00,2013-03-28,3.666667,5.334722,5.000000,3.332778,4.434444,0.666667,0.000000,1.333333,0.0,0.000000,0.0,3.332778
2,u00,2013-03-29,4.666667,5.333333,4.333333,1.333611,2.494444,3.666667,0.666667,1.000000,0.0,0.666667,0.0,1.001389


In [17]:
print(result_2['time_spent_morning_firstplace'].max())
print(result_2['time_spent_afternoon_firstplace'].max())
print(result_2['time_spent_evening_firstplace'].max())
print(result_2['time_spent_night_firstplace'].max())

print(result_2['time_spent_morning_secondplace'].max())
print(result_2['time_spent_afternoon_secondplace'].max())
print(result_2['time_spent_evening_secondplace'].max())
print(result_2['time_spent_night_secondplace'].max())

print(result_2['time_spent_morning_thirdplace'].max())
print(result_2['time_spent_afternoon_thirdplace'].max())
print(result_2['time_spent_evening_thirdplace'].max())
print(result_2['time_spent_night_thirdplace'].max())

5.980833333333333
5.990833333333334
5.99
5.9927777777777775
5.666944444444445
5.833333333333333
5.863888888888889
5.970277777777778
4.666111111111111
5.681388888888889
5.833333333333333
5.833333333333333


In [18]:
df = pd.merge(result_1, result_2, on=['uid', 'date'])
df.to_csv('location_features_studentlife_first_half.csv')